In [1]:
# Importar librerias y configurar logging
import pandas as pd
from pathlib import Path
import tabula
from unidecode import unidecode
import logging
logging.basicConfig(level=logging.DEBUG, filename='log.log',  format='%(asctime)s %(levelname)s %(name)s %(message)s')
logger = logging.getLogger(__name__)

In [2]:
# Listar archivos pdf
mi_path = 'fichas_simp/'
lista_pdfs = list(Path(mi_path).glob('**/*.pdf'))
nombres_pdfs = [pdf.name.replace('.pdf', '') for pdf in lista_pdfs]

In [ ]:
# Extraer tablas
tablas = []
for pdf in lista_pdfs:
    try:
        tabla = tabula.read_pdf(pdf, pages='all', guess=False, lattice=True)
        tablas.append(tabla)
    except Exception as e:
        tablas.append(None)
        logger.error(f'Error al extraer tablas de {pdf}')
        logger.error(e)
        continue

In [3]:
# read pickle
tablas = pd.read_pickle('out/v2/fichas.pkl')

In [4]:
# Extraer y limpiar tablas
tablas_item = []
tablas_adjudicado = []

for idx, tabla in enumerate(tablas):
    try:
        tabla_item = tabla[4] if tabla[4].columns.str.contains('UNSPS').any() else tabla[5] if tabla[5].columns.str.contains('UNSPS').any() else tabla[6]
        tabla_item.columns = tabla_item.columns.str.replace('\r', '', regex=True).str.replace(' ', '').to_series().apply(unidecode)
        tabla_item.drop(tabla_item.columns[-1], axis=1, inplace=True)
        tabla_item.replace('\r', ' ', regex=True, inplace=True)
        tabla_item.drop(tabla_item.index[-1], inplace=True)
        tabla_item.insert(0, 'cuce', nombres_pdfs[idx])
        tabla_item.insert(1, 'list_n', idx)
        tablas_item.append(tabla_item)
    except Exception as e:
        tablas_item.append(None)
        logger.error(f'Error al procesar tablas de ITEMS de la lista: {idx}: {nombres_pdfs[idx]}')
        logger.error(e)
        continue
    try:
        tabla_adjudicado = tabla[3] if tabla[3].columns.str.contains('Nombre proponente').any() else tabla[4] if tabla[4].columns.str.contains('Nombre proponente').any() else tabla[5]
        tabla_adjudicado.insert(0, 'cuce', nombres_pdfs[idx])
        tabla_adjudicado.insert(1, 'list_n', idx)
        tabla_adjudicado.replace('\r', ' ', regex=True, inplace=True)
        tablas_adjudicado.append(tabla_adjudicado)
    except Exception as e:
        tablas_adjudicado.append(None)
        logger.error(f'Error al procesar tablas de ADJUDICADOS la lista: {idx}: {nombres_pdfs[idx]}')
        logger.error(e)
        continue

    

In [5]:
# Unir y exportar tablas a CSV
pd.concat(tablas_item, ignore_index=True).to_csv('out/v1/tablas_item.csv', index=False)


In [8]:
tablas_adjudicado_clean = pd.concat(tablas_adjudicado, ignore_index=True)
tablas_adjudicado_clean.columns = ['cuce', 'list_n', 'numero', 'nombre_proponente', 'monto_adjudicado', 'nro_de', 'monto_de', 'fecha_1', 'monto', 'fecha_2', 'rupe', 'estado']
tablas_adjudicado_clean.to_csv('out/v1/tablas_adjudicado.csv', index=False)

In [ ]:
import pickle
with open('../../fichas/fichas_nuevas.pkl', 'wb') as f:
    pickle.dump(tablas, f)

with open('../../fichas/lista_pdfs.pkl', 'wb') as f:
    pickle.dump(lista_pdfs, f)

In [ ]:
len(lista_pdfs) == len(tablas) == len(tablas_clean)

# Pruebas

In [ ]:
lista_pdfs[35]

In [ ]:
tablas[35][5]